In [297]:
import pymongo

In [298]:
def mongo_connect_build(db_name, col_name):
    
    global mycol
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") 

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    mycol = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立

def data_find():
    # 尋找多筆資料
    return mycol.find({},{'_id':0})

In [299]:
def mongo_connect(db_name, col_name):
    
    global col
    client = pymongo.MongoClient("mongodb://192.168.1.25:27017/") 

    db = client[db_name] # 選擇使用的db,不存在則會在資料輸入時自動建立
    col = db[col_name] # 選擇collection,不存在則會在資料輸入時自動建立

def job_find():
    # 尋找多筆資料
    return col.find()

In [300]:
mongo_connect_build('cakeresume', 'NoEng_resume')

In [301]:
cv_data = data_find()

In [302]:
import re

In [303]:
def isEnglish(s):  # 檢查字元是否為英文
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [304]:
def string_clean_cv(resume):  # 字串清洗
    '''
    input: a string (original resume)
    output: a string (clean resume)
    '''
    job_desc = resume.split('\n')  # 根據換行符號轉乘 List格式
    job_words = ''

    for words in job_desc:
        words = re.sub(r'[^\w\s]', ' ', words)  # remove all punctuations
        words = re.sub(r'\d+', '', words)  # remove all numbers
        words = words.strip()  # remove white space
        job_words += words

    return (job_words)

In [305]:
def jieba_cut_CV(data, stop_words): # 定義使用結巴程式
    '''
    input:
        data : string
        stop_words: list of stopwords
    output:
        str_result : a string after jieba (job content after clean and cut)
    '''

    # 使用結巴斷詞，產生 list of words
    seg_result = jieba.cut(data, cut_all=False)

    # 篩選斷詞，去掉單一中文字
    lst_seg = []

    for i in list(seg_result):
        i = i.strip()
        if len(i) < 1:  # 排除空值
            continue
        elif isEnglish(i) == False and len(i) == 1:  # 排除單一中文字
            continue
        elif i.isdigit() == True:  # 排除數字
            continue
        elif i in stop_words:  # 排除stopwords
            continue

        else:
            lst_seg.append(i)
            
        str_result = ' '.join(lst_seg)

    return str_result

In [306]:
import jieba
import json

In [307]:
jieba.load_userdict('./Jobcontent_dict.txt')  # 指定辭典檔

# 指定 Stop words檔案
with open(file='./Jobcontent_stopwords.txt', mode='r', encoding="UTF-8") as file:
    stop_words = file.read().split('\n')
    stop_words = [i.strip() for i in stop_words]

In [308]:
# 從JSON檔找到儲存內容欄位

lst_CV = []
lst_id = []

for i in cv_data:
    data_id = i['id']
    data = i['value']
    if isEnglish(data) == False:
        lst_CV.append(data)
        lst_id.append(data_id)

In [309]:
# CV資料清洗
lst_CV_clean = []

for i in lst_CV:
    data = string_clean_cv(i)
    data = jieba_cut_CV(data, stop_words)
    lst_CV_clean.append(data)

In [310]:
CV_clean = []

for i, j in list(zip(lst_id, lst_CV_clean)):
    cv_dict = {}
    cv_dict['id'] = i
    cv_dict['value'] = [i for i in j.split(' ')]
    CV_clean.append(cv_dict)


In [319]:
from sklearn.feature_extraction.text import CountVectorizer
import pickle

In [320]:
test_cv = lst_CV_clean

In [321]:
 # 將結巴後的斷詞，利用CountVectorizer轉換減少字詞數量
def CountVectorizer_transform(content):
    content = [content]
    vectorizer = pickle.load(open("vectorizer_5000.pickel", "rb")) 
    transform_content = vectorizer.transform(content)
    result = vectorizer.inverse_transform(transform_content)
    result = result[0].tolist()
    
    return result

In [322]:
# word2vec的list中的list為一個文本,故每個履歷jieba後丟進一個小的list,創造w2v_list為整個要訓練的list
def resume_w2v_list(test):
    w2v_list = []
    for i in test:
        result = CountVectorizer_transform(i)
        w2v_list.append(result)
    return w2v_list

In [323]:
w2v_list = resume_w2v_list(test_cv)

In [324]:
#總共有610個履歷
len(w2v_list)

610

In [325]:
# 查看每個履歷jieba後有多少個詞
for i in w2v_list:
    print(len(i))

226
236
131
171
184
184
165
156
124
137
212
101
145
96
109
97
100
110
89
82
250
206
283
116
77
159
233
88
131
158
177
205
222
226
109
115
184
258
326
95
199
156
113
79
136
182
89
199
221
163
72
74
220
21
369
267
80
80
107
180
104
129
101
109
120
217
142
210
129
132
169
165
162
243
93
260
185
149
125
191
89
205
165
81
148
231
93
124
229
179
218
200
132
100
105
221
122
281
186
129
76
152
164
149
144
106
134
215
169
135
138
257
89
102
96
100
183
113
105
144
137
154
69
134
118
199
41
60
167
111
104
97
183
158
190
344
133
212
146
47
180
90
82
221
135
116
138
127
90
91
120
156
119
56
120
154
136
126
115
80
73
66
63
156
150
75
121
393
148
80
195
132
148
338
124
207
244
117
95
97
109
172
130
401
147
69
172
133
110
174
117
119
170
131
61
88
215
102
168
147
165
107
129
114
98
52
151
109
233
159
71
185
272
195
141
93
66
180
131
123
112
116
137
104
159
142
191
118
114
161
120
158
245
51
344
143
170
78
78
173
125
139
64
140
153
202
65
142
97
108
127
214
141
52
197
111
166
125
100
105
139
147
185
22

In [326]:
from gensim.models import Word2Vec
import pandas as pd
import time

In [327]:
# word2vec建模
def word2vec_model_train(train_list):
    start = time.time()
    num_features = 100  
    # Word vector dimensionality - The size of the dense vector to represent each token or word 
    #(i.e. the context or neighboring words). If you have limited data, then size should be a much smaller value 
    #since you would only have so many unique neighbors for a given word

    min_word_count = 1                        
    # Minimium frequency count of words. The model would ignore words that do not satisfy the min_count.

    num_workers = 10       # How many threads to use behind the scenes?

    context = 200          
    # The maximum distance between the target word and its neighboring word.  
    # 查看一個履歷有多少個詞用意在於window要多大
    
    downsampling = 1e-2   
    # threshold for configuring which higher-frequency words are randomly downsampled

    # Initialize and train the model 
    model = Word2Vec(train_list, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context,sample = downsampling, iter=10,sg=1)

    # If you don't plan to train the model any further, calling 
    # init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)

    end = time.time()
    return model
    print(f'Training completed. It cost : {round(end-start,2)} sec')

In [328]:
resume_model4 = word2vec_model_train(w2v_list)

In [354]:
resume_model4.save('resume_model_1030_4.model')

In [329]:
len(resume_model4.wv.vocab)

4005

In [330]:
# 找職缺測試
mongo_connect('jieba_clean', 'job_clean_final')
job_data = job_find().limit(10)

In [331]:
# 創建job_list來測試模型
def create_job_list(job_data):
    job_list = []
    for i in job_data:
        content_list = []
        for j in i['jobDescription_clean']:
            content_list.append(j)
        job_list.append(content_list)
    return job_list
    

In [332]:
job_list = create_job_list(job_data)

In [333]:
job_list[0]

['介紹',
 '銷售',
 '商品',
 '顧客',
 '接待',
 '服務',
 '電話',
 '諮詢',
 '調貨',
 '包裝',
 '退換貨',
 '商品',
 '進貨',
 '入庫',
 '銷售',
 '管理',
 '庫存',
 '管理',
 '商品',
 '包裝',
 '陳列',
 '促銷品',
 '換檔',
 '維持',
 '店櫃',
 '週遭',
 '整潔',
 '提升',
 '業績',
 '淑女',
 '服飾',
 '銷售',
 '服務',
 '門市',
 '店員',
 '專櫃',
 'Windows',
 'Outlook']

In [334]:
# 要將job_list中的詞清為model中有訓練到的詞,否則會噴錯
def create_joblist_for_similar(resume_jieba,model):
    job_list_all = []
    for word_list in resume_jieba:
        worker_list= []
        for word in word_list:
            # model.wv.vocab.keys()為dict,可查看模型有訓練到的詞
            if word in model.wv.vocab.keys():
                worker_list.append(word)
            else:
                pass
        job_list_all.append(worker_list)
    return job_list_all

In [335]:
job_list = create_joblist_for_similar(job_list)

In [336]:
mongo_connect_build('cakeresume', 'NoEng_resume')
cv_data = data_find()

In [337]:
# 加回每個cv的id
cv_keys = []
for i in cv_data:
    cv_keys.append(i['id'])
dict_prob = {}
for k,v in zip(cv_keys,w2v_list):
    dict_prob[k] = v

In [355]:
dict_prob

{'yo': ['analysis',
  'aoi',
  'app',
  'application',
  'azure',
  'bootstrap',
  'cc',
  'codeigniter',
  'communication',
  'computing',
  'cpr',
  'css',
  'data',
  'django',
  'education',
  'es',
  'ex',
  'experience',
  'gcp',
  'gept',
  'hadoop',
  'html',
  'ibm',
  'ic',
  'inventor',
  'iot',
  'javascript',
  'jquery',
  'js',
  'lab',
  'microsoft',
  'mining',
  'ml',
  'mobile',
  'mysql',
  'node',
  'php',
  'project',
  'python',
  'sas',
  'school',
  'server',
  'skills',
  'spark',
  'sql',
  'streaming',
  'toeic',
  'vue',
  'web',
  '交通',
  '人工智慧',
  '企業',
  '作品',
  '信託',
  '健身',
  '偵測',
  '優化',
  '元件',
  '兒童',
  '全國',
  '全球',
  '公會',
  '兼任',
  '分析',
  '前後端',
  '前端',
  '創新',
  '功能',
  '助人',
  '助教',
  '助理',
  '半導體',
  '協助',
  '協調',
  '台灣',
  '同業',
  '吧台',
  '品質',
  '商業',
  '國家',
  '國際',
  '外包',
  '大地',
  '大學',
  '大數據',
  '學校',
  '學生',
  '客戶',
  '客服',
  '客製化',
  '家教',
  '實習',
  '實習生',
  '實驗',
  '實驗室',
  '封裝',
  '專案',
  '專業',
  '專題',
  '工具',
  '工業',
  '工研院',
  '

In [356]:
def open_json_file(CACHE_FNAME):
    try:
        cache_file = open(CACHE_FNAME, 'r', encoding='utf-8-sig')
        cache_contents = cache_file.read()
        CACHE_DICTION = json.loads(cache_contents, encoding='utf-8-sig')
        cache_file.close()
        return CACHE_DICTION

    except:
        CACHE_DICTION = {}
        return CACHE_DICTION
    
def dump_json_file(query_dict, file_name):
    dumped_json_cache = json.dumps(query_dict)
    fw = open(file_name,"w")
    fw.write(dumped_json_cache)
    fw.close()
    print('successfully write down the file: ', file_name)    

In [357]:
dump_json_file(dict_prob,'resume_for_similar.json')

successfully write down the file:  resume_for_similar.json


In [350]:
'''
丟入要匹配的user_list,
用model.n_similarity相似度匹配,
用user_list配對每一筆cv_list,求出相似度建立字典
'''
def job_map_cv(cv_dict,model,user_list)
    rate_dict = {}
    for k,v in zip(cv_dict.keys(),cv_prob.values()):    
        rate = model.n_similarity(v,user_list)
        rate_dict[k] = rate

c:\users\big data\pycharmprojects\pyetl\venv\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: Call to deprecated `n_similarity` (Method will be removed in 4.0.0, use self.wv.n_similarity() instead).
  This is separate from the ipykernel package so we can avoid doing imports until


In [351]:
#依相似度排列,由高到低
rate_sort = sorted(rate_dict.items(), key=lambda x:x[1],reverse=True)

In [353]:
similar_list[5]

['咖啡',
 '飲品',
 '調製',
 '外場',
 '餐飲',
 '服務',
 '接待',
 '文書處理',
 '作業',
 '協助',
 '咖啡',
 '吧檯',
 '餐飲',
 '服務',
 '空間',
 'concept',
 '餐飲',
 '服務生']

In [352]:
list(rate_sort)[0:10]

[('lan-arther', 0.915225),
 ('aabsguy452', 0.91166127),
 ('qe5863995', 0.89657414),
 ('werdxc2015', 0.8960696),
 ('henry2320', 0.88233215),
 ('taylor0917', 0.88180673),
 ('194250', 0.8815332),
 ('andy_lai', 0.8808965),
 ('kkman00234', 0.87414104),
 ('jordanxooi', 0.873374)]

In [296]:
list(rate_sort)[0:10]

[('4b12d4', 0.9038497),
 ('sherrylin789', 0.8921007),
 ('ppm0330', 0.8903691),
 ('fmvy5513', 0.8879225),
 ('24a556', 0.88756305),
 ('myra-liu', 0.8870198),
 ('loveskymore', 0.8863758),
 ('johnson', 0.8858237),
 ('wendy-fu', 0.88290805),
 ('sarahhsu', 0.88246006)]